In [1]:
import pandas as pd
import math
import networkx
import os
from parsel import Selector

In [21]:
df = pd.read_csv("../spider/data.csv",encoding='utf-8',index_col="title")
path = "../spider/htmls"
files = os.listdir(path)

In [50]:
# 存储一个网页链接到一个列表的映射，列表中的元素为链接对应HTML文档中可以跳转到的链接
url_dict = {}

In [53]:
for file in files:
    title = str(file).split(".html")[0]
    url = df.loc[title].url
    with open(os.path.join(path,file),'r',encoding='utf-8-sig') as f :
        text = f.read()
        selector = Selector(text)
        url_list = []
        url_list.extend(selector.css('a::attr(href)').getall())
        url_dict[url] = url_list

In [62]:
# 计算pageRank：
def get_page_rank():
    # 创建一个有向图
    digraph = networkx.DiGraph()
    # 在有向图中添加有向边
    for url, url_list in url_dict.items():
        for url2 in url_list:
            if url2 in df.url.values:
                digraph.add_edge(url, url2)
    # 引入阻尼因子，利用库函数计算pagerank
    pagerank = networkx.pagerank(digraph, alpha=0.85)
    page_rank_df = pd.Series(pagerank, name='page_rank')
    # 对 PageRank 进行处理，缩放并限制数值范围，避免权重过大
    page_rank_df = page_rank_df.apply(lambda x: math.log(x * 10000, 10) + 1)
    page_rank_df.to_csv("./page_rank.csv", encoding='utf-8-sig')

In [63]:
get_page_rank()

In [8]:
# 查看排名靠前的一些网页：
page_rank = pd.read_csv("page_rank.csv",encoding='utf-8')
page_rank.sort_values(by='page_rank',ascending=False).head(10)

,url,page_rank
1,http://news.nankai.edu.cn/index.shtml,3.643053
2,http://news.nankai.edu.cn/ywsd/index.shtml,3.643053
3,http://news.nankai.edu.cn/mtnk/index.shtml,3.643053
4,http://news.nankai.edu.cn/gynk/index.shtml,3.643053
5,http://news.nankai.edu.cn/nkrw/index.shtml,3.643053
6,http://news.nankai.edu.cn/nkdxb/index.shtml,3.643053
7,http://news.nankai.edu.cn/sp/index.shtml,3.643053
8,http://news.nankai.edu.cn/gb/index.shtml,3.643053
13,http://news.nankai.edu.cn/ywsd/system/2023/11/...,3.623492
18,http://news.nankai.edu.cn/ywsd/system/2023/11/...,3.623492
